In [2]:
import pandas as pd
import glob
import os
import re
import math
import matplotlib.pyplot as plt
import numpy as np
from statistics import mean


In [112]:
def metric(path, r_string, type_string, timing=False):
    resultdf = dict()

    all_files = glob.glob(os.path.join(path, "*"+r_string+type_string+"*.csv"))
    for file in all_files:
        if os.path.getsize(file) == 0:
            all_files.remove(file)
 
    for gnn_metric in all_files:
        lf = gnn_metric.split("/")[-1]
        lf = lf.split(".")[0]
        name = lf.split("__"+type_string+"__")[0]
        if name not in resultdf:
            resultdf[name] = dict()
        df = pd.read_csv(gnn_metric,index_col=None,header=0)
        max_key = df.tail(1).index.values[0]
        gnn_iteration = df['Iteration'][max_key]
        gnn_cpd = df['CPD (ns)'][max_key]
        gnn_time = sum(df['Time'])
        gnn_width = df[' Channel Width'][max_key]

        resultdf[name]['GNN_Iters'] = gnn_iteration
        resultdf[name]['GNN_CPD (ns)'] = gnn_cpd
        resultdf[name]['GNN_Time (s)'] = gnn_time
        resultdf[name]['GNN_Width'] = gnn_width

    all_files = glob.glob(os.path.join(path, r_string+"*reg*.csv"))
    for file in all_files:
        if os.path.getsize(file) == 0:
            all_files.remove(file)

    type_string = "reg"
    for reg_metric in all_files:
        lf = reg_metric.split("/")[-1]
        lf = lf.split(".")[0]
        name = lf.split("__"+type_string+"__")[0]
        if name not in resultdf:
            resultdf[name] = dict()
        df = pd.read_csv(reg_metric,index_col=None,header=0)
        max_key = df.tail(1).index.values[0]
        reg_iteration = df['Iteration'][max_key]
        reg_cpd = df['CPD (ns)'][max_key]
        reg_time = sum(df['Time'])
        reg_width = df[' Channel Width'][max_key]

        resultdf[name]['REG_Iters'] = reg_iteration
        resultdf[name]['REG_CPD (ns)'] = reg_cpd
        resultdf[name]['REG_Time (s)'] = reg_time
        resultdf[name]['REG_Width'] = reg_width

    
   
    resultdf = pd.DataFrame(resultdf).T
    
    # resultdf['GNN_Iters'] = resultdf['GNN_Iters'].astype(int)
    # resultdf['REG_Iters'] = resultdf['REG_Iters'].astype(int)
    # resultdf['REG_Width'] = resultdf['REG_Width'].astype(int)
    # resultdf['GNN_Width'] = resultdf['GNN_Width'].astype(int)
    
    resultdf['Difference'] = resultdf['REG_Width'] - resultdf['GNN_Width']
    # resultdf['Difference'] = resultdf['Difference'].astype(int)
    resultdf['Difference (%)'] = resultdf['Difference']/resultdf['REG_Width']*100
    resultdf = resultdf[["REG_Iters", "GNN_Iters", 
                        #  "REG_Width", "GNN_Width", 
                         "REG_CPD (ns)", "GNN_CPD (ns)",
                        "Difference"
                         ]]
    pd.set_option('display.float_format','{:.2f}'.format)
    print(resultdf)
    # print(resultdf['Difference (%)'].describe())

In [109]:
path = "/mnt/e/benchmarks/route_metrics/MCNC/"
r_string = "EArch*"
type_string = "gnn"
metric(path, r_string, type_string)
# pd.options.display.float_format = '{:,.0f}'.format
# graph = pd.read_csv("/mnt/e/benchmarks/edges_nodes.csv",index_col=0,header=0)
# print(graph.describe())


                 REG_Iters  GNN_Iters  Difference
EArch__alu4          17.00      19.00       30.00
EArch__apex2         36.00      -1.00       20.00
EArch__apex4         22.00      34.00       12.00
EArch__bigkey        17.00      24.00        0.00
EArch__clma          45.00      -1.00       18.00
EArch__des           55.00      36.00       14.00
EArch__diffeq         8.00      33.00       16.00
EArch__dsip          16.00       9.00        4.00
EArch__elliptic      47.00      -1.00       24.00
EArch__ex1010        52.00      -1.00       22.00
EArch__ex5p          27.00      81.00       22.00
EArch__frisc         41.00      51.00       20.00
EArch__misex3        24.00     138.00       22.00
EArch__pdc           48.00      -1.00       26.00
EArch__s298          23.00      25.00        8.00
EArch__s38417        11.00      34.00        8.00
EArch__s38584        43.00      12.00       14.00
EArch__seq           30.00      -1.00       26.00
EArch__spla          45.00     108.00       32.00


In [113]:
import tensorflow as tf
import datetime

In [116]:
path = "/mnt/e/benchmarks/route_metrics/TITAN/"
r_string = "strat*"
type_string = "gnn"
metric(path, r_string, type_string)

                          REG_Iters  GNN_Iters  REG_CPD (ns)  GNN_CPD (ns)  \
stratixiv_arch__alu4          51.00      69.00         10.13         10.77   
stratixiv_arch__apex2         52.00      63.00         10.32         10.46   
stratixiv_arch__apex4         80.00      63.00          9.71         10.08   
stratixiv_arch__bigkey         7.00       8.00          8.21          8.29   
stratixiv_arch__clma          42.00      76.00          9.75          9.91   
stratixiv_arch__des           42.00      39.00         12.25         12.28   
stratixiv_arch__diffeq        49.00      50.00          7.41          7.55   
stratixiv_arch__dsip          19.00      83.00          8.08          8.11   
stratixiv_arch__ex1010       166.00     126.00         11.29         12.13   
stratixiv_arch__ex5p         105.00     117.00          9.90         10.26   
stratixiv_arch__frisc         41.00      48.00         12.05         12.15   
stratixiv_arch__misex3        42.00      93.00          9.84    